## DIA 057: Creación de un API REST con Flask y SQLAlchemy para Gestión de Usuarios


Hoy crearemos una API RESTful en Flask con SQLAlchemy para manejar usuarios en una base de datos.

✅ ¿Qué aprenderemos?

📌 CRUD de usuarios con Flask.
📌 Uso de SQLAlchemy como ORM.
📌 Validación de datos con Flask Marshmallow.
📌 Respuestas en formato JSON.
✅ Endpoints de la API:

GET /users → Obtener lista de usuarios.
GET /users/<id> → Obtener usuario específico.
POST /users → Crear un nuevo usuario.
PUT /users/<id> → Actualizar usuario existente.
DELETE /users/<id> → Eliminar usuario.
🖥️ Código Completo (app.py)
python
Copiar
Editar
import os
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_marshmallow import Marshmallow

# Configuración de Flask
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///users.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de Base de Datos y Serialización
db = SQLAlchemy(app)
ma = Marshmallow(app)

# ---------------------------
# Modelo de Usuario
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)

# ---------------------------
# Esquema para Serialización
# ---------------------------
class UserSchema(ma.SQLAlchemyAutoSchema):
    class Meta:
        model = User

user_schema = UserSchema()
users_schema = UserSchema(many=True)

# ---------------------------
# Rutas de la API
# ---------------------------
@app.route('/users', methods=['GET'])
def get_users():
    """Obtener todos los usuarios."""
    users = User.query.all()
    return jsonify(users_schema.dump(users))

@app.route('/users/<int:user_id>', methods=['GET'])
def get_user(user_id):
    """Obtener un usuario por ID."""
    user = User.query.get(user_id)
    if not user:
        return jsonify({"error": "Usuario no encontrado"}), 404
    return jsonify(user_schema.dump(user))

@app.route('/users', methods=['POST'])
def create_user():
    """Crear un nuevo usuario."""
    data = request.json
    if not data.get("name") or not data.get("email"):
        return jsonify({"error": "Nombre y email requeridos"}), 400

    new_user = User(name=data["name"], email=data["email"])
    db.session.add(new_user)
    db.session.commit()
    
    return jsonify(user_schema.dump(new_user)), 201

@app.route('/users/<int:user_id>', methods=['PUT'])
def update_user(user_id):
    """Actualizar usuario existente."""
    user = User.query.get(user_id)
    if not user:
        return jsonify({"error": "Usuario no encontrado"}), 404

    data = request.json
    user.name = data.get("name", user.name)
    user.email = data.get("email", user.email)

    db.session.commit()
    return jsonify(user_schema.dump(user))

@app.route('/users/<int:user_id>', methods=['DELETE'])
def delete_user(user_id):
    """Eliminar un usuario."""
    user = User.query.get(user_id)
    if not user:
        return jsonify({"error": "Usuario no encontrado"}), 404

    db.session.delete(user)
    db.session.commit()
    return jsonify({"message": "Usuario eliminado"}), 200

# ---------------------------
# Inicializar la Base de Datos
# ---------------------------
@app.before_first_request
def create_tables():
    db.create_all()

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
🚀 Cómo Probar la API
1️⃣ Instalar dependencias:

sh
Copiar
Editar
pip install flask flask-sqlalchemy flask-marshmallow marshmallow
2️⃣ Ejecutar el Servidor Flask:

sh
Copiar
Editar
python app.py
3️⃣ Probar la API con cURL o Postman:

✅ Obtener usuarios:

sh
Copiar
Editar
curl -X GET http://127.0.0.1:5000/users
✅ Crear usuario:

sh
Copiar
Editar
curl -X POST http://127.0.0.1:5000/users \
     -H "Content-Type: application/json" \
     -d '{"name": "Juan Pérez", "email": "juan@example.com"}'
✅ Actualizar usuario:

sh
Copiar
Editar
curl -X PUT http://127.0.0.1:5000/users/1 \
     -H "Content-Type: application/json" \
     -d '{"name": "Juan Actualizado"}'
✅ Eliminar usuario:

sh
Copiar
Editar
curl -X DELETE http://127.0.0.1:5000/users/1
🔍 Explicación de las Principales Implementaciones
🔹 📂 Uso de SQLAlchemy para Base de Datos

db.Model define el modelo User con atributos name y email.
db.create_all() inicializa la base de datos al iniciar la app.
🔹 📜 Serialización de Datos con Flask Marshmallow

UserSchema convierte los objetos en JSON válido para la API.
Se usa user_schema.dump(user) y users_schema.dump(users).
🔹 🔄 Endpoints CRUD
✅ GET /users → Devuelve todos los usuarios.
✅ GET /users/<id> → Devuelve un usuario por ID.
✅ POST /users → Crea un nuevo usuario.
✅ PUT /users/<id> → Actualiza un usuario existente.
✅ DELETE /users/<id> → Elimina un usuario.

🔹 🚀 Validación de Datos

Se valida que name y email sean requeridos en POST.
Se usa get("name", user.name) en PUT para actualización parcial.